In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
from pymongo import MongoClient

In [28]:
client = MongoClient('localhost', 27017)
db = client['BD2']
hits = db.sales_hits

def write_to_mongo(elems):
    for elem in elems:
        name = elem.find_element_by_class_name('sel-product-tile-title').text
        price = elem.find_element_by_class_name('c-pdp-price__current').text
        hits.insert_one({'name':name, 'price':price})

chrome_options = Options()
chrome_options.add_argument('start-maximized')

driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.mvideo.ru/')

block = driver.find_element_by_class_name('sel-hits-block')
elems = block.find_elements_by_class_name('gallery-list-item')
write_to_mongo(elems)

buttons = block.find_element_by_class_name('carousel-paging').find_elements_by_tag_name('a')[1:]
for button in buttons:
    button.click()
    time.sleep(2)
    block = driver.find_element_by_class_name('sel-hits-block')
    elems = block.find_elements_by_class_name('gallery-list-item')[:4]
    write_to_mongo(elems)